In [ ]:
from API_utils import fetch_from_API, show_dates, show_files_for_site_date
import os
import numpy as np
import json
import multiprocessing
import time
import glob
import rasterio

ncores = multiprocessing.cpu_count()
ncores

16

For now we will just do TEAKs oldest Lidar flight

In [14]:
site = 'TEAK'
productcode = 'DP1.30003.001'
data_path = '/home/jovyan/tmp'

os.makedirs(data_path, exist_ok=True)

#show_dates(site, productcode)


In [19]:
t0 = time.time()

In [20]:
time.time()-t0

15.632129430770874

In [25]:
def generate_laz_download_info():
    '''Returns: time of url issueance, list of laz files'''
    t0 = time.time()
    files = show_files_for_site_date(productcode, site, '2018-06')
    laz = []
    for file in files:
        if 'classified_point_cloud_colorized.laz' in file['name']:
            laz.append(file)
    return(t0, laz)
    
    
def refresh_url(f, t0):
    '''If too much time has elapsed since url issued, modifies f to contain new url'''
    if time.time() - t0 < 3550:
        pass
    else:
        files = show_files_for_site_date(productcode, site, '2018-06')
        for file in files:
            if file['name'] == f['name']:
                f['url'] = file['url']

In [4]:
import urllib
import requests
import hashlib


def download_from_NEON_API(f, data_path):

    attempts = 0 
    while attempts < 4:
        try:
            # get the file 
            handle = requests.get(f['url'])
            
            #check the md5 if it exists
            if f['md5']:
                md5 = hashlib.md5(handle.content).hexdigest()
                if md5 == f['md5']:
                    success = True
                    attempts = 4
                else:
                    fmd5 = f['md5']
                    print(f'md5 mismatch on attempt {attempts}')
                    success = False
                    attempts = attempts + 1
            else: 
                success = True
                attempts = 4
        except Exception as e:
            print(f'Warning:\n{e}')
            success = False
            attempts = attempts + 1
    # write the file
    if success:
        fname = os.path.join(data_path, f['name'])
        with open(fname, 'wb') as sink:
            sink.write(handle.content)
    else:
        raise Exception('failed to download')

In [5]:
#download_from_NEON_API(boundaries, data_path)

We need to isntall pdal to process the laz files

!pip install pdal

Ug, now have to ```conda update numpy``` to use pdal. Need to pin these versions down

In [6]:
import pdal
from string import Template
import subprocess
import time

In [32]:
def make_pipe(f, bbox, out_path, resolution=1):
    tile = '_'.join(f.rpartition('/')[2].split('_')[4:6])
    '''Creates, validates and then returns the pdal pipeline
    
    Arguments:
    bbox       -- Tuple - Bounding box in srs coordintes (default srs is EPSG:3857),
                  in the form: ([xmin, xmax], [ymin, ymax]).
    outpath   -- String - Path where the CHM shall be saved. Must include .tif exstension.
    srs        -- String - EPSG identifier for srs  being used. Defaults to EPSG:3857
                  because that is what ept files tend to use.
    threads    -- Int - Number os threads to be used by the reader.ept. Defaults to 4.
    resolution -- Int or Float - resolution (m) used by writers.gdal
    '''
    
    t = Template('''
    {
        "pipeline": [
            {
            "filename": "${f}",
            "type": "readers.las",
            "tag": "readdata"
            },
            {
            "type":"filters.outlier",
            "method":"radius",
            "radius":1.0,
            "min_k":4
            },
            {
            "type":"filters.range",
            "limits":"Classification[3:5]"
            },
            {
            "type":"filters.optimalneighborhood",
            "min_k":8,
            "max_k": 50
            },
            {
            "type":"filters.covariancefeatures",
            "knn":10,
            "threads": 2,
            "feature_set": "all"
            },
            {
            "filename": "${outpath}/${tile}_Anisotropy.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Anisotropy",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_DemantkeVerticality.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "DemantkeVerticality",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Density.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Density",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Eigenentropy.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Eigenentropy",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Linearity.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Linearity",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Omnivariance.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Omnivariance",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Planarity.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Planarity",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Scattering.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Scattering",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_EigenvalueSum.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "EigenvalueSum",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_SurfaceVariation.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "SurfaceVariation",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Verticality.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Verticality",
            "bounds": "${bbox}"
            }
        ]
    }''')

    pipe = t.substitute(f=f, bbox=bbox, outpath=out_path, tile=tile, resolution=resolution)
    pipeline = pdal.Pipeline(pipe)
    if pipeline.validate():
        return(pipeline, tile)
    else:
        raise Exception('Bad pipeline (sorry to be so ambigous)!')



    Anisotropy

    DemantkeVerticality

    Density

    Eigenentropy

    Linearity

    Omnivariance

    Planarity

    Scattering

    EigenvalueSum

    SurfaceVariation

    Verticality


In [48]:
def make_hyper_lidar_tif(f, data_path, t0, verbose=False, vverbose=False):
    
    # Contemplate the verbosity
    if vverbose: verbose = True
        
    # make sure url is still valid
    refresh_url(f, t0)
    
    # name of file to be stored
    name = os.path.join(data_path, f['name'])
    size = f['size']
    
    # Download the laz
    download_from_NEON_API(f, data_path)
    if verbose: print('Download completed.')
   
    # find the bounds
    cmd = f'pdal info {name}'
    if verbose: print(f'About to call {cmd}')
    reply = subprocess.run(cmd, shell=True, capture_output=True)
    if vverbose:print(f'stdout was:\n\n{reply.stdout}\n---------------------------')
    meta = json.loads(reply.stdout)
    if vverbose: print(f'The json looks like:\n\n {meta}\n---------------------------')
    bbox = meta['stats']['bbox']['native']['bbox']
    bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']])
    if verbose: print(f'Bounds are:\n{bounds}\n')
    
    # make and execute the pdal pipeline
    pipeline, tile = make_pipe(name, bounds, data_path, resolution=1)
    count = pipeline.execute()
    
    # remove the laz file
    os.remove(name)
    
    # get and sort the layers to stack
    layers = [item for item in os.listdir(data_path) if tile in item]
    layers.sort()
    if vverbose: print(f'Layers are:\n{layers}')
        
    # make tags for the bands    
    tags = [l.rpartition('_')[2].split('.')[0] for l in layers]
    if verbose: print(f'Tags are:\m{tags}')
    
    # Read metadata of first layer
    lyr = os.path.join(data_path, layers[0])
    with rasterio.open(lyr) as src0:
        meta = src0.meta

    # Update meta to reflect the number of layers
    meta.update(count = len(layers))

    # Read each layer and write it to stack
    with rasterio.open(f'lidar_stack_{tile}.tif', 'w', **meta) as dst:
        for id, layer in enumerate(layers, start=1):
            lyr = os.path.join(data_path, layer)
            with rasterio.open(lyr) as src1:
                dst.write_band(id, src1.read(1))
        
        for id, tag in enumerate(tags, start=1):
            dst.update_tags(id, ColorInterp=tag)
            
    for layer in layers:
        os.remove(os.path.join(data_path, layer))

    return(count)

In [49]:
t0, laz = generate_laz_download_info()

for f in laz[:1]:
    make_hyper_lidar_tif(f, data_path, t0, verbose=True)

Download completed.
About to call pdal info /home/jovyan/tmp/NEON_D17_TEAK_DP1_320000_4099000_classified_point_cloud_colorized.laz
Bounds are:
([320000, 320999.999], [4099000, 4099999.999])

Tags are:\m['Anisotropy', 'DemantkeVerticality', 'Density', 'Eigenentropy', 'EigenvalueSum', 'Linearity', 'Omnivariance', 'Planarity', 'Scattering', 'SurfaceVariation', 'Verticality']


In [40]:
l = '320000_4099000_Eigenentropy.tif'
l.rpartition('_')[2].split('.')[0]


'Eigenentropy'

In [10]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

t0, laz = generate_laz_download_info()

results = []
for f in laz[:5]:
    results.append(delayed(make_hyper_lidar_tif)(f, data_path, t0))

with ProgressBar():
    computed = compute(*results)

[########                                ] | 20% Completed | 22.1s


KeyboardInterrupt: 

In [1]:
data_path

NameError: name 'data_path' is not defined

In [98]:
f = laz[3]
download_from_NEON_API(f, data_path)
name = os.path.join(data_path, f['name'])

In [99]:
cmd = f'pdal info {name}'
reply = subprocess.run(cmd, shell=True, capture_output=True)
reply

CompletedProcess(args='pdal info /home/jovyan/tmp/NEON_D17_TEAK_DP1_321000_4093000_classified_point_cloud_colorized.laz', returncode=1, stdout=b'', stderr=b"PDAL: readers.las: File signature is not 'LASF', is this an LAS/LAZ file?\n\n")

In [88]:

reply = subprocess.run(cmd, shell=True, capture_output=True)
meta = json.loads(reply.stdout)
bbox = meta['stats']['bbox']['native']['bbox']
bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)